<a href="https://colab.research.google.com/github/juansebastiangit/Inteligencia_artificial/blob/main/Taller_regresores_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 1 TAM - Regresores

*Ejercicio 1 - Demostración